## Gradio Deployement

In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
from torchvision import transforms
import gradio as gr

#Define Transforms to be applied to input image
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#identify device available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

#Image Size
IMAGE_SIZE = (224,224)

#set the saved Model path
modelpath = 'deeplabv3resnet101_ep42_valscore0.753.pth'
#Load Model
model = torch.load(modelpath,map_location=torch.device(device))
#Set in Eval mode
model.eval()

#Define function the creates the predictions and converts as binary image to be displayed as output
def predict(image):
    image = cv2.resize(image,IMAGE_SIZE)
    image = image_transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        generatemask = model(image)['out']
        prediction = np.squeeze(generatemask.cpu().numpy())
        threshold = 0.5
        binary_mask = (prediction >= threshold).astype(np.uint8) * 255
        binary_image = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
       
    return binary_image

#Create the Gradio UI

title = '<u>Car Scratch Detection using Binary Mask</u>'
description='<h4>Car Scratch Detection using a tuned pretained DeepLabV3 model</h4> <h4>Upload a car picture</h4>'
demo = gr.Interface(fn=predict,title=title, description=description, inputs=gr.Image(shape=(224, 224)),outputs=gr.Image(shape=(224, 224),label="Predicted Mask"),allow_flagging='never')

#launch the web app
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
